Example of how to use the OrderParameterEnsemble class to calculate nematic order parameters, translational order parameters and structure factors. The example system consists of a coarse grain simulation of water and monoglyceride solution (more details can be found in "files_orderparams/README.md"). 

The first step is to import the ClusterEnsemble and OrderParameterEnsemble classes.

In [2]:
from clustercode.ClusterEnsemble import ClusterEnsemble
from clustercode.OrderParameterEnsemble import OrderParameterEnsemble

The next step is to load the relevant files:
    - run.tpr: contains the simulation run information
    - init.gro: is the starting simulation configuration
    - traj.xtc: is the simulation trajectory
These can then be used to initiliase an instance of the OrderParameterEnsemble class

In [4]:
tpr  = "files_orderparams/run.tpr"
init_config = "files_orderparams/init.gro"
traj = "files_orderparams/traj.xtc"


OrderParamEns = OrderParameterEnsemble(tpr, traj, ['MGE'])

First we calculate the nematic order parameters of the initial configuration. The supplied trajectory file (files_orderparams/traj.xtc) has all the atoms in the box (gromacs flag -pbc atom), the molecules need to be made whole to do the nematic order parameter analysis thus we supply the parameter pbc_style='mol', which will use the gromacs command gmx trjconv to convert the file into the desired format (alternatively this could be done externally and a trajectory file with the gromacs flag -pbc mol could be supplied).

We see how the nematic order parameter changes from the first configuration (time: 0 ns) to the final configuration (time: 500 ns). This is expected as at time 0 ns the system is ordered randomly, whereas at time 500 ns it forms a lamellar phase. This can be seen visually, in files_orderparams run the command: vmd init.gro traj.xtc -e vmdlog.

In [6]:
OrderParamEns.nematic_op_analysis(times=[0], pbc_style='mol', style='molecule')

OrderParamEns.nematic_op_analysis(times=[500000], pbc_style='mol', style='molecule')




****TIME:     0.00
Nematic order parameter: 0.182
****MEAN:
Mean nematic order parameter: 0.182 +/- 0.000
Mean system director: [-0.30158747  0.91599136  0.26458424]
****TIME: 500000.00
Nematic order parameter: 0.526
****MEAN:
Mean nematic order parameter: 0.526 +/- 0.000
Mean system director: [ 0.319589   -0.94156166  0.10641669]


The nematic_op_analysis() method assigns a number of attributes:

In [9]:
print(OrderParamEns.nematic_op_list)
print(OrderParamEns.system_director_list)
print(OrderParamEns.mean_nematic_op, OrderParamEns.stdev_nematic_op)
print(OrderParamEns.mean_system_director)

[0.5299535166100975]
[array([-0.31757436,  0.94224242, -0.10642247])]
0.5299535166100975 0.0
[-0.31757436  0.94224242 -0.10642247]


The nematic order parameter calculations depend on a principal molecular axis that is chosen for each molecule (or group of atoms). The two options available in the code are:
    - The principal interntial axis (default, used above)
    - The end-to-end vector uses the vector between the first and last atom in each atom group
As can be seen below for the cases of the monoglyceride (MGE) molecules using the end-to-end vector are small compared to the principal inertial axis.

In [8]:
OrderParamEns.nematic_op_analysis(times=[0], pbc_style='mol', style='molecule', principal_axis='end-to-end')

OrderParamEns.nematic_op_analysis(times=[500000], pbc_style='mol', style='molecule', principal_axis='end-to-end')

****TIME:     0.00
Nematic order parameter: 0.186
****MEAN:
Mean nematic order parameter: 0.186 +/- 0.000
Mean system director: [-0.30667707  0.9113041   0.27472534]
****TIME: 500000.00
Nematic order parameter: 0.530
****MEAN:
Mean nematic order parameter: 0.530 +/- 0.000
Mean system director: [-0.31757436  0.94224242 -0.10642247]


Get clusters to test whether structure analysis works on custom trajectories

ClusEnsemble = ClusterEnsemble(tpr, traj, ['CM','CE'])

ClusEnsemble.cluster_analysis(cut_off=7.5, times=(450000,500000), work_in="Residue", style="atom", traj_pbc_style="mol", pbc=False)

for frame in ClusEnsemble.cluster_list:
    print(len(frame))
    print([len(clus) for clus in frame])